In [29]:
import numpy as np 
import matplotlib.pyplot as plt
from dsepruning import skel_pruning_DSE
import os
import cv2 as cv 
from os.path import join
from skimage import io, morphology
from skimage.morphology import medial_axis, skeletonize, dilation
from scipy.ndimage import distance_transform_edt
from skimage import filters
import progressbar 
%matplotlib qt

In [30]:
#path of the images
path = r'D:\Dalhousie\datasets\DrilledSphere'
folder = r'2023.10.24 DrilledSphere' 

#Path for saving the results
saving_folder = r'skelnotdilated'

in_path=join(path,folder)
entries = os.listdir(in_path)
image_path = join(in_path,entries[0]) #Selected image
savedimage_path = join(path,saving_folder)

In [31]:
#Visualization of image in the specified path
img=cv.imread(image_path,0)
plt.figure()
plt.imshow(img,cmap='gray')

#If there is measures and text in the image, a masking is need it 
mask = np.zeros(img.shape,dtype=np.uint8)
mask[80:641,359:912]=255 
img[mask==0]=0

cv.imshow('Imagen original',img)
cv.waitKey(0)

-1

In [32]:
#Function to create a frequency grid to create the filter.
def Grid2D(imag_shape):
    ysize,xsize = imag_shape #The grid needs to be the same size as the image
    ymid = ysize // 2
    xmid = xsize // 2
    
    if ysize % 2 == 0: #Condition to led with images with odd or even size.
        ymax = ymid - 1
    else:
        ymax = ymid

    if xsize % 2 == 0:
        xmax = xmid - 1
    else:
        xmax = xmid
    
    X,Y= np.meshgrid(np.arange(-xmid, xmax + 1),np.arange(-ymid, ymax + 1)) #Spatial domain of the grid
    X = np.fft.ifftshift(X)/xsize #Frecuency domain of the grid (also normalized)
    Y = np.fft.ifftshift(Y)/ysize
    
    return X,Y

# Riez Filter

The monogenic signal framework extends the analytic signal concept to 2D by the introduction of a vector valued odd filter (Riesz filter) whose Fourier domain representation is:

\begin{equation}
(H_1(u,v),H_2(u,v)) = \left(i\frac{u}{\sqrt{u^2+v^2}} , i\frac{v}{\sqrt{u^2+v^2}} \right)
\end{equation}

# Quadrature Filter
# LogGabor Filter

\begin{equation}
G(\omega) = exp\left(-\frac{(log(\frac{\omega}{\omega_0})^2}{2(log(\frac{k}{\omega_0})^2}\right)
\end{equation}

Thus, the monogenic signal in the spatial domain is constructed as:

\begin{equation}
f_M = \begin{bmatrix}
f(x,y)*g(x,y)\\
f(x,y)*g(x,y)*h_1(x,y)\\
f(x,y)*g(x,y)*h_2(x,y) 
\end{bmatrix}
\end{equation}

And in frequency domain: 

\begin{equation}
F_M = \begin{bmatrix}
\mathscr{F}[f(x,y)]\cdot[G_\omega]\\
\mathscr{F}[f(x,y)]\cdot[G_\omega]\cdot H_1(u,v)\\
\mathscr{F}[f(x,y)]\cdot[G_\omega]\cdot H_2(u,v)
\end{bmatrix} = \begin{bmatrix}
F_{M1}\\
F_{M2}\\
F_{M3}
\end{bmatrix} =  \begin{bmatrix}
even\\
odd_1\\
odd_2
\end{bmatrix}
\end{equation}

Thus, the even and the odd can be represented by: 

\begin{equation}
even_{MG}(x,y) = f_{M1}(x,y)
\end{equation}

and 

\begin{equation}
odd_{MG}(x,y) = \sqrt{f_{M2}^2(x,y)+f_{M3}^2(x,y)}
\end{equation}

Rajpoot, et al, metion that obtaining the real part of the odd signals, then a analytical signal can be constructed as: 

\begin{equation}
F_{MG} = even_{MG}(x,y)+i*odd_{MG}(x,y)
\end{equation}

An important thing to mention is that by applying it, loses the local orientation infromation, but since we are focousedin local phase information, which ramains unaffected by the proposal. 

Then, is posible to define the Feature Asymmetry measure with: 

\begin{equation}
FA(x,y) = \frac{\lfloor |odd_{MG}(x,y)| - |even_{MG}(x,y)| - T \rfloor }{\sqrt{even_{MG}^2(x,y)+ odd_{MG}^2(x,y)}}
\end{equation}

And the Feature Symmetry measure is expressed as: 

\begin{equation}
FS(x,y) = \frac{\lfloor |even_{MG}(x,y)|-|odd_{MG}(x,y)| - T \rfloor }{\sqrt{even_{MG}^2(x,y)+ odd_{MG}^2(x,y)}}
\end{equation}

Where T is a threshold, and is given by: 

\begin{equation}
T = exp^{log\left(mean\left(\sqrt{even_{MG}^2(x,y)+ odd_{MG}^2(x,y)}\right)\right)}
\end{equation}

In [33]:
def RiezFilter(img_shape): 
    u,v = Grid2D(img_shape)
    w = np.sqrt(u**2+v**2)
    w[0,0]=0.0001
    
    H1 = (1j * u)/ w
    H2 = (1j * v)/ w
    
    return H1,H2

def LogGaborFilter_MS(img_shape,scaling_factor,wavelenght,k_w0):
    
    #Create a Frequency Grid
    X,Y = Grid2D(img_shape)
    
    FiltBank = np.zeros((len(wavelenght),img_shape[0],img_shape[1]))
    
    #Determine the spatial regions to use 
    w = np.sqrt(Y**2+X**2)
    w[0,0]=0.0001 #Avoids log(0)
    phi = np.arctan2(Y,X)
    
    #User defined parameter
    m = scaling_factor
    
    for i in range(len(wavelenght)):
        
        lamb = wavelenght[i]
        w_0 = 2/(lamb*3)

        GabFilter = np.exp( -((np.log(w/w_0))**2 / (2*(np.log(k_w0))**2))) 
        GabFilter[0,0] = 0

        if (img_shape[0]%2 == 0):
            GabFilter[img_shape[0]//2,:]=0

        if (img_shape[1]%2 == 0):
            GabFilter[:,img_shape[1]//2]=0

        FiltBank[i,:,:] = GabFilter
        
    return FiltBank

def MonogenicSignal(image,LogGaborFilt,H1,H2):
    
    F = np.fft.fft2(image)
    Ffilt = np.multiply(F,LogGaborFilt)
    
    F_g = np.fft.ifft2(Ffilt)
    fm1 = np.real(F_g)
    
    F_modd_1 = np.multiply(Ffilt,H1) 
    F_modd_1 = np.fft.ifft2(F_modd_1)
    fm2 = np.real(F_modd_1)
    #print(np.min(np.imag(F_modd_1)))
    
    F_modd_2 = np.multiply(Ffilt,H2) 
    F_modd_2 = np.fft.ifft2(F_modd_2)
    fm3 = np.real(F_modd_2)
    
    return fm1,fm2,fm3


def FeatureSymetry_MS(m1,m2,m3):
    epsilon = 0.01
    
    odd = np.sqrt(m2**2 + m3**2)
    even = np.abs(m1)
    
    T = np.exp(np.log(np.mean(np.sqrt(odd**2+even**2))))
    
    denominator = np.sqrt(even**2+odd**2)+epsilon
    
    FS_num = np.clip((even - odd - T), 0,None)
    FA_num = np.clip((odd - even - T), 0,None)
    
    FS = FS_num/denominator
    FA = FA_num/denominator
    
    FS = np.sum(FS, axis=0)
    FA = np.sum(FA, axis=0)
    
    
    return FS,FA

# Bone Post Processing

In [34]:
def BonePostProcessing(img): 
    I = img.copy()
    
    kernel = cv.getStructuringElement(cv.MORPH_RECT,(3,3))
    Im = I>0
    
    #Dilation of the objects in the image and labeling
    Im = cv.dilate(I*255,kernel,iterations = 1)
    Arr = Im>0
    labeled,num_objects = morphology.label(Arr, return_num = True)
    
    #Skeletonize and pruning
    skel, dist = medial_axis(Arr, return_distance=True)
    new_skel = skel_pruning_DSE(skel, dist, 1000)
    Cleaned = new_skel

    #labeling after skeletonization
    labeled,num_objects = morphology.label(Cleaned, return_num = True)
    
    #Obtaining areas 
    areas = np.zeros(num_objects)
    for objects in range (num_objects):
        areas[objects] = len(labeled[labeled==objects])
    
    #Obtaining the largest areas
    arguments = areas.argsort()[-5:][::-1]
    arguments = arguments[1:]

    
    Zeros = np.zeros(Cleaned.shape,dtype=np.uint8) 
    
    #Measuring the depth (centroid in Y coordinate) in the image based on the Moments of the convex hull
    cY_prev = Zeros.shape[0]//2 #Condition to eliminate soft tissue noise in the first iteration
    
    if arguments.shape[0] > 0:
        for i in range (len(arguments)):
            mask = np.zeros(Cleaned.shape,dtype=np.uint8)
            mask[labeled == arguments[i]] = 255
            
            convex_hull = morphology.convex_hull_object(mask,connectivity = 2)
            x_coordinates, y_coordinates = np.where(convex_hull)
            M = cv.moments(np.c_[y_coordinates,x_coordinates]) #Obtaining the moments of the convex hull
            cY_c = np.abs(int(M["m01"] / (M["m00"]+.0001))) #Centroid in Y
            
            #With the following block, we make sure to keep the detected objects with 
            #the centroid located at greater depth. We need to apply dilation again in order to 
            #connect components if those are disconected, then we need to skeletonize and prune the skeleton again
            if cY_c>(cY_prev): 
                Zeros[labeled == arguments[i]] = 255

                cY_prev = cY_c          
                
                Zeros = cv.dilate(Zeros,kernel,3)
                Arr = Zeros > 0 

                Skel = morphology.remove_small_holes(Arr,500)
                Skel = morphology.skeletonize(Skel)
                dist2 = distance_transform_edt(Arr,return_indices=False,return_distances=True)
                Pruned = skel_pruning_DSE(Skel,dist2,350)

                Zeros = Zeros*0
                Zeros[Pruned != 0] = 255 
    
    Zeros = cv.dilate(Zeros,kernel,iterations = 2)
    
    
    return Zeros


# Sphere Processing

Does not require a lot of processing as the ultrasound videos

In [35]:
def SpherePostProcessing(img): 
    I = img.copy()
    
    kernel = cv.getStructuringElement(cv.MORPH_RECT,(3,3))
    Im = I>0
    Im = cv.dilate(I*255,kernel,iterations = 1)
    Arr = Im>0
    
    REMOVED = morphology.remove_small_holes(Arr,1000)
    
    labeled,num_objects = morphology.label(REMOVED, return_num = True)    
    Cleaned = labeled
    
    areas = np.zeros(num_objects)
    for objects in range (num_objects):
        areas[objects] = len(labeled[labeled==objects])
    
    arguments = areas.argsort()[-5:][::-1]
    arguments = arguments[1:]

    Zeros = np.zeros(Cleaned.shape,dtype=np.uint8)
    Zeros2 = np.zeros(Cleaned.shape,dtype=np.uint8)
    
    #Measuring the depth (centroid in Y coordinate) in the image based on the Moments of the convex hull
    
    cY_prev = 0
    
    if arguments.shape[0] > 0:
        for i in range (len(arguments)):
            mask = np.zeros(Cleaned.shape,dtype=np.uint8)
            mask[labeled == arguments[i]] = 255
            
            convex_hull = morphology.convex_hull_object(mask,connectivity = 2)
            
            
            x_coordinates, y_coordinates = np.where(convex_hull)
            
            M = cv.moments(np.c_[y_coordinates,x_coordinates])
            cY_c = np.abs(int(M["m01"] / (M["m00"]+.0001)))
            
            #With the following block, we make sure to keep the detected objects with 
            #the centroid located at greater depth. We need to apply dilation again in order to 
            #connect components if those are disconected, then we need to skeletonize and prune the skeleton again
            if cY_c>(cY_prev):
                Zeros[labeled == arguments[i]] = 255
                cY_prev = cY_c          

                Zeros = cv.dilate(Zeros,kernel,3)
                Arr = Zeros > 0 

                Skel = morphology.remove_small_holes(Arr,5000)
                Skel = morphology.skeletonize(Skel)
                dist2 = distance_transform_edt(Arr,return_indices=False,return_distances=True)
                Pruned = skel_pruning_DSE(Skel,dist2,350)
                
                Zeros = Zeros*0
                Zeros[Pruned != 0] = 255 
    

    return Zeros

In [36]:
# Call this function to run all the Feature Asymmetry calculation and Post Processing
def Allprocess(I,num_frame):
    
    img = I.copy()
    H1,H2 = RiezFilter(img.shape)
    
    #Empirically, we've detected that the wavelenghts that works for the sphere are wl =np.array([50,55,60])
    #and for the bone wl = np.array([3,6,9,15,30,36])
    wl = np.array([50,55,60]) 

    
    lG = LogGaborFilter_MS(img.shape,1.5,wl,0.15)

    m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)


    FS_m,FA_m = FeatureSymetry_MS(m1,m2,m3)

    FA_m = (FA_m-FA_m.min())*(255/(FA_m.max()-FA_m.min()))
    FA_m = FA_m.astype(np.uint8)
    FS_m = (FS_m-FS_m.min())*(255/(FS_m.max()-FS_m.min()))
    FS_m = FS_m.astype(np.uint8)
    
    #Post processing
    Cleaned_Skel = SpherePostProcessing(FS_m)    
    
    #Visualization
    FS_M = FS_m.copy()
    FS_M = cv.cvtColor(FS_M, cv.COLOR_GRAY2BGR)
    Rc = cv.cvtColor(I,cv.COLOR_GRAY2BGR)
    
    Rc[Cleaned_Skel[:,:] == 255] = (0,255,0)  
    I_c = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
    Results = np.concatenate((I_c,FS_M,Rc),axis=1)  #Comparision of the Original Image, 
                                                    #the Feature Symmetry resulted image 
                                                    #and the detected bone surface in the original image
    
    return Results , FS_M, Rc, Cleaned_Skel

# Video Processing

In [10]:

cap = cv.VideoCapture('D:/Dalhousie/datasets/Extracorporeal/Videos/image_12936930724906.mp4')
import astropy.units as u

c = 0
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    Comparision,FS,I = Allprocess(gray,c)

    shape = frame.shape
    
    
    cv.imshow('Feature Symmetry',FS)
    cv.imshow('Bone Highlighted',I)
    
    c += 1

    if cv.waitKey(1) == ord('q'):
        break
    else:
        cv.waitKey(1)

cap.release()

# Image Processing

In [38]:
import progressbar
maximumvalue = len(entries)-10
bar = progressbar.ProgressBar(maxval=maximumvalue).start()

for ith in range (maximumvalue-1,maximumvalue):
    
    in_image_path = join(in_path,entries[ith])
    out_image_path = join(savedimage_path,entries[ith])

    img = cv.imread(in_image_path,0)
    
    Comparision,FS,Bone_Surface, Sekeleton= Allprocess(img[33:629,256:767],ith)
    cv.imshow('Comparision',Comparision)
    cv.imshow('Feature Symmetry',FS)
    cv.imshow('Bone Highlighted',Bone_Surface)
    cv.imshow('Sekeleton',Sekeleton)
    cv.waitKey(0)
    bar.update(ith)